In [1]:
%load_ext autoreload
%autoreload
from IPython.display import clear_output

import os
import sys
import subprocess
import pdb
import time
import datetime
import math
import random
import _pickle as cPickle
from collections import defaultdict

from six.moves import zip_longest
import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow import distributions as tfd
from tensorflow.keras.preprocessing.sequence import pad_sequences

from data_structure import get_batches, get_test_batches
from components import tf_log, sample_latents, compute_kl_losses, dynamic_rnn, dynamic_bi_rnn

from topic_beam_search_decoder import BeamSearchDecoder

In [2]:
PAD = '<pad>' # This has a vocab id, which is used to pad the encoder input, decoder input and target sequence
UNK = '<unk>' # This has a vocab id, which is used to represent out-of-vocabulary words
BOS = '<p>' # This has a vocab id, which is used at the beginning of every decoder input sequence
EOS = '</p>' # This has a vocab id, which is used at the end of untruncated target sequences

# load data & set config

In [3]:
def del_all_flags(FLAGS):
    flags_dict = FLAGS._flags()    
    keys_list = [keys for keys in flags_dict]    
    for keys in keys_list:
        FLAGS.__delattr__(keys)

del_all_flags(tf.flags.FLAGS)

flags = tf.app.flags

flags.DEFINE_string('gpu', '1', 'visible gpu')

flags.DEFINE_string('mode', 'train', 'set train or eval')

flags.DEFINE_string('data_path', 'data/bags/instances.pkl', 'path of data')
flags.DEFINE_string('modeldir', 'model/topic_vae', 'directory of model')
flags.DEFINE_string('modelname', 'bags', 'name of model')

flags.DEFINE_integer('epochs', 50, 'epochs')
flags.DEFINE_integer('batch_size', 64, 'number of sentences in each batch')
flags.DEFINE_integer('log_period', 1000, 'valid period')

flags.DEFINE_string('opt', 'Adagrad', 'optimizer')
flags.DEFINE_float('lr', 0.05, 'lr')
flags.DEFINE_float('reg', 1., 'regularization term')
flags.DEFINE_float('grad_clip', 5., 'grad_clip')

flags.DEFINE_float('keep_prob', 0.8, 'dropout rate')
flags.DEFINE_float('word_keep_prob', 0.75, 'word dropout rate')

flags.DEFINE_bool('warmup', True, 'flg of warming up')
flags.DEFINE_integer('epochs_cycle', 5, 'number of epochs within a cycle')
flags.DEFINE_float('r_cycle', 0.5, 'proportion used to increase beta within a cycle')
flags.DEFINE_integer('warmup_topic', 0, 'warmup period for KL of topic')

flags.DEFINE_integer('beam_width', 2, 'beam_width')
flags.DEFINE_float('length_penalty_weight', 0.0, 'length_penalty_weight')

flags.DEFINE_integer('n_topic', 20, 'number of topic')
flags.DEFINE_integer('dim_hidden_bow', 256, 'dim of hidden bow')
flags.DEFINE_integer('dim_latent_bow', 32, 'dim of latent topic')
flags.DEFINE_integer('dim_emb', 256, 'dim_emb')
flags.DEFINE_integer('dim_hidden', 512, 'dim_hidden')
flags.DEFINE_integer('dim_hidden_topic', 512, 'dim_hidden_topic')
flags.DEFINE_integer('dim_latent_topic', 32, 'dim_latent_topic')
flags.DEFINE_bool('bidirectional', True, 'flg of bidirectional encoding')

# for evaluation
flags.DEFINE_string('refdir', 'ref', 'refdir')
flags.DEFINE_string('outdir', 'out', 'outdir')

flags.DEFINE_string('f', '', 'kernel')
flags.DEFINE_bool('logtostderr', True, 'kernel')
flags.DEFINE_bool('showprefixforinfo', False, '')
flags.DEFINE_bool('verbosity', False, '')
# flags.DEFINE_integer('stderrthreshold', 20, 'kernel')

config = flags.FLAGS

flags.DEFINE_string('modelpath', os.path.join(config.modeldir, config.modelname), 'path of model')

In [4]:
os.environ["CUDA_VISIBLE_DEVICES"] = config.gpu

In [5]:
instances_train, instances_dev, instances_test, word_to_idx, idx_to_word, bow_idxs = cPickle.load(open(config.data_path,'rb'))

In [6]:
train_batches = get_batches(instances_train, config.batch_size)
dev_batches = get_batches(instances_dev, config.batch_size)
test_batches = get_test_batches(instances_test, config.batch_size)

In [7]:
flags.DEFINE_integer('PAD_IDX', word_to_idx[PAD], 'PAD_IDX')
flags.DEFINE_integer('UNK_IDX', word_to_idx[UNK], 'UNK_IDX')
flags.DEFINE_integer('BOS_IDX', word_to_idx[BOS], 'BOS_IDX')
flags.DEFINE_integer('EOS_IDX', word_to_idx[EOS], 'EOS_IDX')

flags.DEFINE_integer('n_vocab', len(word_to_idx), 'n_vocab')
flags.DEFINE_integer('dim_bow', len(bow_idxs), 'dim_bow')

maximum_iterations = max([max([instance.max_sent_l for instance in batch]) for ct, batch in dev_batches])
flags.DEFINE_integer('maximum_iterations', maximum_iterations, 'maximum_iterations')

flags.DEFINE_integer('cycle_steps', len(train_batches)*config.epochs_cycle, 'number of steps for each cycle')

In [8]:
def debug_shape(variables):
    sample_batch = dev_batches[0][1]
    feed_dict = get_feed_dict(sample_batch)
    _variables = sess.run(variables, feed_dict=feed_dict)
    for _variable, variable in zip(_variables, variables):
        if hasattr(variable, 'name'):
            print(variable.name, ':', _variable.shape)
        else:
            print(_variable.shape)

def debug_value(variables, return_value=False):
    sample_batch = test_batches[0][1]
    feed_dict = get_feed_dict(sample_batch)
    _variables = sess.run(variables, feed_dict=feed_dict)

    if return_value: 
        return _variables
    else:
        for _variable, variable in zip(_variables, variables):
            if hasattr(variable, 'name'):
                print(variable.name, ':', _variable)
            else:
                print(_variable)
                
def check_shape(variables):
    if 'sess' in globals(): raise
    sess = tf.Session()
    sess.run(tf.global_variables_initializer())
    
    sample_batch = test_batches[0][1]
    feed_dict = get_feed_dict(sample_batch)
    _variables = sess.run(variables, feed_dict=feed_dict)
    for _variable, variable in zip(_variables, variables):
        if hasattr(variable, 'name'):
            print(variable.name, ':', _variable.shape)
        else:
            print(_variable.shape)
            
    sess.close()
    
def check_value(variables):
    if 'sess' in globals(): raise
    sess = tf.Session()
    sess.run(tf.global_variables_initializer())
    
    sample_batch = test_batches[0][1]
    feed_dict = get_feed_dict(sample_batch)
    _variables = sess.run(variables, feed_dict=feed_dict)
    for _variable, variable in zip(_variables, variables):
        if hasattr(variable, 'name'):
            print(variable.name, ':', _variable)
        else:
            print(_variable.shape)
            
    sess.close()    
    

# run model 

## init

In [9]:
def get_feed_dict(batch, mode='train'):
    bow = np.array([instance.bow for instance in batch]).astype(np.float32)
    keep_prob = config.keep_prob if mode == 'train' else 1.0
    feed_dict = {
                t_variables['bow']: bow, 
                t_variables['keep_prob']: keep_prob
    }
    return  feed_dict

In [10]:
tf.reset_default_graph()

t_variables = {}
t_variables['bow'] = tf.placeholder(tf.float32, [None, config.dim_bow])
t_variables['keep_prob'] = tf.placeholder(tf.float32)

tree_idxs = {0:[1, 2], 
             1:[10, 11], 2:[20, 21, 22], 
             10: [100, 101], 11: [110, 111, 112], 20: [200, 201], 21: [210, 211], 22:[220, 221, 222]
             }

# tree_idxs = {0:[1, 2], 
#                       1:[10, 11], 2:[20, 21, 22]}

topic_idxs = [0] + [idx for child_idxs in tree_idxs.values() for idx in child_idxs]
child_to_parent_idxs = {child_idx: parent_idx for parent_idx, child_idxs in tree_idxs.items() for child_idx in child_idxs}

## doubly rnn

In [11]:
class DoublyRNNCell:
    def __init__(self, dim_hidden, output_layer=None):
        self.dim_hidden = dim_hidden
        
        self.ancestral_layer=tf.layers.Dense(units=dim_hidden, activation=tf.nn.tanh, name='ancestral')
        self.fraternal_layer=tf.layers.Dense(units=dim_hidden, activation=tf.nn.tanh, name='fraternal')
        self.hidden_layer = tf.layers.Dense(units=dim_hidden, activation=tf.nn.tanh, name='hidden')
        
        self.output_layer=output_layer
        
    def __call__(self, state_ancestral, state_fraternal, reuse=True):
        with tf.variable_scope('input', reuse=reuse):
            state_ancestral = self.ancestral_layer(state_ancestral)
            state_fraternal = self.fraternal_layer(state_fraternal)

        with tf.variable_scope('output', reuse=reuse):
            state_hidden = self.hidden_layer(state_ancestral + state_fraternal)
            if self.output_layer is not None: 
                output = self.output_layer(state_hidden)
            else:
                output = state_hidden
            
        return output, state_hidden
    
    def get_initial_state(self, name):
        initial_state = tf.get_variable(name, [1, self.dim_hidden], dtype=tf.float32)
        return initial_state
    
    def get_zero_state(self, name):
        zero_state = tf.zeros([1, self.dim_hidden], dtype=tf.float32, name=name)
        return zero_state

In [12]:
def doubly_rnn(dim_hidden, tree_idxs, initial_state_parent=None, initial_state_sibling=None, output_layer=None, name=''):
    outputs, states_parent = {}, {}
    
    with tf.variable_scope(name, reuse=False):
        doubly_rnn_cell = DoublyRNNCell(dim_hidden, output_layer)

        if initial_state_parent is None: initial_state_parent = doubly_rnn_cell.get_initial_state('init_state_parent')
        if initial_state_sibling is None: 
#             initial_state_sibling = doubly_rnn_cell.get_initial_state('init_state_sibling')
            initial_state_sibling = doubly_rnn_cell.get_zero_state('init_state_sibling')
        output, state_sibling = doubly_rnn_cell(initial_state_parent, initial_state_sibling, reuse=False)
        outputs[0], states_parent[0] = output, state_sibling

        for parent_idx, child_idxs in tree_idxs.items():
            state_parent = states_parent[parent_idx]
            state_sibling = initial_state_sibling
            for child_idx in child_idxs:
                output, state_sibling = doubly_rnn_cell(state_parent, state_sibling)
                outputs[child_idx], states_parent[child_idx] = output, state_sibling

    return outputs

## stick break

In [13]:
def hierarchical_sbp(tree_sticks_topic, tree_sticks_branch):
    tree_prob_topic = {}
    rest_topics = {}

    # calculate topic probability and save
    stick_topic = tree_sticks_topic[0]
    tree_prob_topic[0] = stick_topic
    rest_topics[0] = 1.-stick_topic
    for parent_idx, child_idxs in tree_idxs.items():
        rest_topic = rest_topics[parent_idx]
        rest_branch = 1.
        for child_idx in child_idxs:
            # calculate topic probability
            if child_idx == child_idxs[-1]: # last child
                prob_branch = rest_branch # phi
            else:
                stick_branch = tree_sticks_branch[child_idx] # psi
                prob_branch = stick_branch * rest_branch # phi

            if not child_idx in tree_idxs: # leaf childs
                prob_topic = prob_branch * rest_topic # pi
            else:
                stick_topic = tree_sticks_topic[child_idx] # upsilon
                prob_topic = stick_topic * prob_branch * rest_topic # pi

            # save topic probability and update rest stick length
            tree_prob_topic[child_idx] = prob_topic
            rest_branch = (1.- stick_branch) * rest_branch
            rest_topics[child_idx] = (1.-stick_topic)*prob_branch * rest_topic
            
    return tree_prob_topic

## build model

In [14]:
# encode bow
with tf.variable_scope('topic/enc', reuse=False):
    hidden_bow_ = tf.layers.Dense(units=config.dim_hidden_bow, activation=tf.nn.relu, name='hidden_bow')(t_variables['bow'])
    hidden_bow = tf.layers.Dropout(t_variables['keep_prob'])(hidden_bow_)
    means_bow = tf.layers.Dense(units=config.dim_latent_bow, name='mean_bow')(hidden_bow)
    logvars_bow = tf.layers.Dense(units=config.dim_latent_bow, kernel_initializer=tf.constant_initializer(0), bias_initializer=tf.constant_initializer(0), name='logvar_bow')(hidden_bow)
    latents_bow = sample_latents(means_bow, logvars_bow) # sample latent vectors
    prob_layer = lambda h: tf.nn.sigmoid(tf.matmul(latents_bow, h, transpose_b=True))
    
    tree_sticks_topic = doubly_rnn(config.dim_latent_bow, tree_idxs, output_layer=prob_layer, name='sticks_topic')
    tree_sticks_branch = doubly_rnn(config.dim_latent_bow, tree_idxs, output_layer=prob_layer, name='sticks_branch')

    tree_prob_topic = hierarchical_sbp(tree_sticks_topic, tree_sticks_branch)
    prob_topic = tf.concat(list(tree_prob_topic.values()), 1)

# decode bow
with tf.variable_scope('shared', reuse=False):
    embeddings = tf.get_variable('emb', [config.n_vocab, config.dim_emb], dtype=tf.float32, initializer=tf.contrib.layers.xavier_initializer()) # embeddings of vocab

bow_embeddings = tf.nn.embedding_lookup(embeddings, bow_idxs) # embeddings of each bow features

with tf.variable_scope('topic/dec', reuse=False):
    tree_hidden_topic_emb = doubly_rnn(config.dim_emb, tree_idxs, name='hidden_topic_emb')
    hidden_topic_emb = tf.concat(list(tree_hidden_topic_emb.values()), 0)
    
    means_topic_emb = tf.layers.Dense(units=config.dim_latent_topic, name='mean_topic_emb')(hidden_topic_emb)
    logvars_topic_emb = tf.layers.Dense(units=config.dim_latent_topic, kernel_initializer=tf.constant_initializer(0), bias_initializer=tf.constant_initializer(0), name='logvar_topic_emb')(hidden_topic_emb)
    latents_topic_emb = sample_latents(means_topic_emb, logvars_topic_emb) # sample latent vectors
    topic_embeddings = tf.layers.Dense(units=config.dim_emb, name='topic_emb')(latents_topic_emb)
    
    topic_bow = tf.nn.softmax(tf.matmul(topic_embeddings, bow_embeddings, transpose_b=True), 1) # bow vectors for each topic
    logits_bow = tf_log(tf.matmul(prob_topic, topic_bow)) # predicted bow distribution

## define loss

In [15]:
# define losses
topic_losses_recon = -tf.reduce_sum(tf.multiply(t_variables['bow'], logits_bow), 1)
topic_loss_recon = tf.reduce_mean(topic_losses_recon) # negative log likelihood of each words

topic_losses_kl_bow = compute_kl_losses(means_bow, logvars_bow) # KL divergence b/w latent dist & gaussian std
topic_loss_kl_bow = tf.reduce_mean(topic_losses_kl_bow, [0]) #mean of kl_losses over batches

means_topic_emb_prior = tf.concat([tf.expand_dims(latents_topic_emb[topic_idxs.index(child_to_parent_idxs[topic_idx])], 0) if topic_idx in child_to_parent_idxs else tf.zeros([1, config.dim_latent_topic], dtype=tf.float32) for topic_idx in topic_idxs], 0)
topic_losses_kl_emb = compute_kl_losses(means_topic_emb, logvars_topic_emb, means_topic_emb_prior)
topic_loss_kl_emb = tf.reduce_sum(topic_losses_kl_emb, [0]) #sum of kl_losses over topics

topic_bow_norm = topic_bow / tf.norm(topic_bow, axis=1, keepdims=True)
topic_dots = tf.clip_by_value(tf.matmul(topic_bow_norm, tf.transpose(topic_bow_norm)), -1., 1.)
topic_loss_reg = tf.reduce_mean(tf.square(topic_dots - tf.eye(len(topic_idxs))))

global_step = tf.Variable(0, name='global_step',trainable=False)

loss = topic_loss_recon + topic_loss_kl_bow + topic_loss_kl_emb + config.reg * topic_loss_reg
# loss = topic_loss_recon + topic_loss_kl_bow + topic_loss_kl_emb

In [16]:
# define optimizer
if config.opt == 'Adam':
    optimizer = tf.train.AdamOptimizer(config.lr)
elif config.opt == 'Adagrad':
    optimizer = tf.train.AdagradOptimizer(config.lr)

grad_vars = optimizer.compute_gradients(loss)
clipped_grad_vars = [(tf.clip_by_value(grad, -config.grad_clip, config.grad_clip), var) for grad, var in grad_vars]
opt = optimizer.apply_gradients(clipped_grad_vars, global_step=global_step)

# monitor
n_bow = tf.reduce_sum(t_variables['bow'], 1)
topic_ppls = tf.divide(topic_losses_recon, tf.maximum(1e-5, n_bow))
topics_freq_bow_indices = tf.nn.top_k(topic_bow, 10, name='topic_freq_bow').indices

In [17]:
def get_loss(sess, batches):
    losses = []
    ppl_list = []
    for ct, batch in batches:
        feed_dict = get_feed_dict(batch, mode='test')
        loss_batch, topic_loss_recon_batch, topic_loss_kl_bow_batch, topic_loss_kl_emb_batch, topic_loss_reg_batch, ppls_batch \
            = sess.run([loss, topic_loss_recon, topic_loss_kl_bow, topic_loss_kl_emb, topic_loss_reg, topic_ppls], feed_dict = feed_dict)
        losses += [[loss_batch, topic_loss_recon_batch, topic_loss_kl_bow_batch, topic_loss_kl_emb_batch, topic_loss_reg_batch]]
        ppl_list += list(ppls_batch)
    loss_mean, topic_loss_recon_mean, topic_loss_kl_bow_mean, topic_loss_kl_emb_mean, topic_loss_reg_mean = np.mean(losses, 0)
    ppl_mean = np.exp(np.mean(ppl_list))
    return loss_mean, topic_loss_recon_mean, topic_loss_kl_bow_mean, topic_loss_kl_emb_mean, topic_loss_reg_mean, ppl_mean

def print_topic_sample():
    topics_freq_bow_idxs = bow_idxs[sess.run(topics_freq_bow_indices)]
    assert len(topics_freq_bow_idxs) == len(topic_idxs)
    for topic_idx, topic_freq_bow_idxs in zip(topic_idxs, topics_freq_bow_idxs):
        print(topic_idx, ' BOW:', ' '.join([idx_to_word[idx] for idx in topic_freq_bow_idxs]))

In [18]:
if 'sess' in globals(): sess.close()
sess = tf.Session()
sess.run(tf.global_variables_initializer())

losses_train = []
ppls_train = []
loss_min = np.inf
beta_eval = 1.
epoch = 0
train_batches = get_batches(instances_train, config.batch_size, iterator=True)
saver = tf.train.Saver(max_to_keep=10)

log_df = pd.DataFrame(columns=pd.MultiIndex.from_tuples(
                    list(zip(*[['','','','TRAIN:','TM','','','', '','VALID:','TM','','','', ''],
                            ['Time','Ep','Ct','LOSS','PPL','NLL','KL(BOW)','KL(EMB)','REG','LOSS','PPL','NLL','KL(BOW)','KL(EMB)','REG']]))))

In [19]:
if len(log_df) == 0:
    cmd_rm = 'rm -r %s' % config.modeldir
    res = subprocess.call(cmd_rm.split())

    cmd_mk = 'mkdir %s' % config.modeldir
    res = subprocess.call(cmd_mk.split())

time_start = time.time()
while epoch < config.epochs:
    for ct, batch in train_batches:
        feed_dict = get_feed_dict(batch)

        _, loss_batch, topic_loss_recon_batch, topic_loss_kl_bow_batch, topic_loss_kl_emb_batch, topic_loss_reg_batch, ppls_batch = \
        sess.run([opt, loss, topic_loss_recon, topic_loss_kl_bow, topic_loss_kl_emb, topic_loss_reg, topic_ppls], feed_dict = feed_dict)
            
        losses_train += [[loss_batch, topic_loss_recon_batch, topic_loss_kl_bow_batch, topic_loss_kl_emb_batch, topic_loss_reg_batch]]
        ppls_train += list(ppls_batch)

        if ct%config.log_period==0:
            loss_train, topic_loss_recon_train, topic_loss_kl_bow_train, topic_loss_kl_emb_train, topic_loss_reg_train = np.mean(losses_train, 0)
            ppl_train = np.exp(np.mean(ppls_train))
            loss_dev, topic_loss_recon_dev, topic_loss_kl_bow_dev, topic_loss_kl_emb_dev, topic_loss_reg_dev, ppl_dev = get_loss(sess, dev_batches)
            global_step_log = sess.run(tf.train.get_global_step())
            
#             if loss_dev < loss_min:
#                 loss_min = loss_dev
#                 saver.save(sess, config.modelpath, global_step=global_step_log)

            clear_output()
    
            time_log = int(time.time() - time_start)
            log_series = pd.Series([time_log, epoch, ct, \
                    '%.2f'%loss_train, '%.0f'%ppl_train, '%.2f'%topic_loss_recon_train, '%.2f'%topic_loss_kl_bow_train, '%.2f'%topic_loss_kl_emb_train, '%.2f'%topic_loss_reg_train, \
                    '%.2f'%loss_dev, '%.0f'%ppl_dev, '%.2f'%topic_loss_recon_dev, '%.2f'%topic_loss_kl_bow_dev, '%.2f'%topic_loss_kl_emb_dev, '%.2f'%topic_loss_reg_dev],
                    index=log_df.columns)
            log_df.loc[global_step_log] = log_series
            display(log_df)
            
            # visualize topic
            print_topic_sample()

            time_start = time.time()
            
    epoch += 1
    train_batches = get_batches(instances_train, config.batch_size, iterator=True)

TRAIN:    TM                                 VALID:  \
      Time Ep    Ct    LOSS   PPL     NLL KL(BOW) KL(EMB)   REG     LOSS   
1        2  0     0  428.37  1041  123.52    0.47  303.46  0.93  1333.77   
1001    27  0  1000  381.49   639  115.00    0.07  265.84  0.58   321.89   
2001    28  0  2000  350.65   603  114.39    0.04  235.66  0.56   281.19   
2276     7  1     0  342.32   599  114.20    0.03  227.54  0.55   271.06   
3276    27  1  1000  320.89   588  113.90    0.02  206.45  0.52   263.56   
4276    27  1  2000  309.17   583  113.71    0.02  194.94  0.49   262.50   
4551     7  2     0  306.85   582  113.68    0.02  192.66  0.49   262.30   
5551    26  2  1000  300.39   579  113.59    0.02  186.32  0.47   262.08   
6551    25  2  2000  295.82   577  113.52    0.01  181.83  0.46   263.65   
6826     7  3     0  294.81   576  113.50    0.01  180.84  0.46   264.93   
7826    25  3  1000  291.67   575  113.49    0.01  177.71  0.45   265.00   
8826    25  3  2000  289.26   573  113.43    0.01  175.38  0.44   262.26   
9101     7  4     0  288.65   573  113.40    0.01  174.80  0.44   261.89   
10101   24  4  1000  286.78   572  113.38    0.01  172.96  0.43   259.99   
11101   25  4  2000  285.23   571  113.35    0.01  171.45  0.43   262.09   
11376    7  5     0  284.86   571  113.34    0.01  171.09  0.42   262.44   
12376   25  5  1000  283.63   570  113.31    0.01  169.89  0.42   262.14   
13376   24  5  2000  282.56   570  113.31    0.01  168.83  0.42   260.07   
13651    7  6     0  282.29   569  113.29    0.01  168.57  0.42   259.98   
14651   24  6  1000  281.41   569  113.30    0.01  167.69  0.41   260.98   
15651   25  6  2000  280.62   568  113.26    0.01  166.94  0.41   261.24   
15926    7  7     0  280.42   568  113.26    0.01  166.75  0.41   262.61   

         TM                                 
        PPL     NLL KL(BOW)  KL(EMB)   REG  
1      1010  115.76    0.33  1216.80  0.88  
1001    551  105.78    0.01   215.55  0.56  
2001    541  105.58    0.00   175.17  0.44  
2276    541  105.55    0.00   165.04  0.47  
3276    541  105.58    0.00   157.56  0.43  
4276    539  105.48    0.00   156.63  0.40  
4551    540  105.51    0.00   156.38  0.41  
5551    539  105.51    0.00   156.17  0.40  
6551    539  105.53    0.00   157.74  0.38  
6826    542  105.60    0.00   158.96  0.38  
7826    540  105.50    0.00   159.14  0.36  
8826    538  105.46    0.00   156.43  0.37  
9101    537  105.44    0.00   156.07  0.38  
10101   537  105.43    0.00   154.18  0.38  
11101   534  105.33    0.00   156.39  0.37  
11376   534  105.30    0.00   156.75  0.38  
12376   536  105.42    0.00   156.35  0.37  
13376   535  105.41    0.00   154.31  0.36  
13651   539  105.49    0.00   154.14  0.35  
14651   534  105.37    0.00   155.25  0.36  
15651   535  105.36    0.00   155.52  0.35  
15926   536  105.37    0.00   156.90  0.34

0  BOW: ! color & bought carry quality perfect pocket pro 'm
1  BOW: price color carry love room ! bought ; 'm perfectly
2  BOW: ! color price ; perfect bought carry - sleeve put
10  BOW: & ! sleeve bought price 'm pocket pockets recommend quality
11  BOW: ! color sleeve nice & quality pocket bought 'm carry
20  BOW: color pockets & sleeve ! ; quality bought inside cover
21  BOW: ; love ! sleeve carry quality bought nice perfect recommend
22  BOW: sleeve ! & love bought pocket perfectly 'm quality recommend
100  BOW: pockets sleeve ! ; perfect cover bought pocket inside strap
101  BOW: carry color love & nice bought cover quality top small
110  BOW: color sleeve ! & room bought 'm recommend - pro
111  BOW: sleeve room nice carry ! bought perfect recommend pro &
112  BOW: & nice carry room love bought quality top made recommend
200  BOW: color carry price ! bought sleeve inside quality 'm pockets
201  BOW: cover room nice sleeve ! ; perfect buy back bought
210  BOW: ; love sleeve pocket

KeyboardInterrupt: 

# confirm variables

In [20]:
debug_value([latents_topic_emb[:, :10]])

strided_slice_19:0 : [[ 0.21753907  0.07204466  0.7631408  -0.26556033  0.3029676  -0.30324614
   0.35927606 -0.9404421  -0.21166287 -0.22711468]
 [-0.88495517 -1.2729504  -0.06644841 -0.16798365 -0.28058505  0.21584143
  -1.007041    0.7131555  -0.16079128  0.19241676]
 [ 0.76782805  0.36915356  0.25338364 -0.6422284   0.40771547 -0.02977864
  -0.76577073  0.07792357  0.06747736 -0.6700013 ]
 [ 0.28741127  0.11993314  0.21345678 -0.28749937  0.40291667  0.77047455
   0.8034781  -1.2522215  -0.94268686  0.49624532]
 [-0.18175046 -0.61139435  0.815388    0.18675652  0.9911157   0.6194456
   0.23734666 -0.8827667   0.2400854   0.8241065 ]
 [ 0.1980838   0.23583126  0.09645793 -0.2298323   0.52760196  0.7409453
  -0.31405747  0.6994357  -1.0257697   0.06848067]
 [ 0.39920264 -0.18473095 -0.58932364  0.29469782  1.1761643  -0.39143768
   0.07835373 -0.58123946  0.77835506 -0.11401667]
 [ 0.6047649  -0.53151464  0.30988187  0.3314864   0.67091465 -0.23454961
   0.62227964  0.11186042 -0.387

In [21]:
debug_value([topic_embeddings[:, :10]])

strided_slice_20:0 : [[ 1.76221579e-01  4.97326702e-02 -2.04024799e-02 -2.69107759e-01
   5.71069494e-02 -3.45422328e-03 -1.44809186e-01 -1.83497116e-01
   2.92611569e-01 -2.02644795e-01]
 [ 1.09006621e-01  8.94176736e-02  1.66517541e-01 -5.61151743e-01
  -1.87023044e-01  3.06396365e-01 -1.92162152e-02 -1.15632989e-01
   4.10124399e-02 -1.07188053e-01]
 [ 1.92586064e-01 -4.65401150e-02 -4.76625450e-02 -2.22691715e-01
  -5.28703332e-02 -3.07369754e-02 -2.45148554e-01  3.90416421e-02
  -2.80977506e-02 -2.76473224e-01]
 [-1.25005960e-01  1.24910668e-01  4.06295173e-02 -6.02804601e-01
   2.10111588e-03  1.89532153e-02  3.23301733e-01 -7.81081617e-02
  -9.07959938e-02 -1.67737633e-01]
 [ 9.95765701e-02 -1.50087141e-02  9.74102467e-02 -5.21605015e-01
   5.22680432e-02  1.72805823e-02 -7.50711188e-02 -6.81969672e-02
   2.91521214e-02 -1.06882699e-01]
 [ 4.10097018e-02  3.41513716e-02 -4.54074778e-02 -6.07545793e-01
  -7.85243064e-02  5.67239858e-02 -6.53014854e-02 -5.91549724e-02
  -3.9551526

In [53]:
debug_value([prob_topic[0]])

strided_slice_20:0 : [1.7139336e-01 1.1444765e-02 5.5537068e-09 1.6592527e-03 3.2894917e-05
 6.8514816e-09 1.5045667e-09 4.7043186e-10 7.0054650e-01 9.1958398e-05
 8.4566280e-02 2.9324554e-02 9.2093315e-04 4.6922491e-06 7.1599129e-11
 1.1960822e-05 3.1369327e-10 2.7913263e-06 2.4284012e-11 7.0988722e-15]


In [40]:
debug_value([prob_topic[3]])

strided_slice_3:0 : [0.2951853  0.09169052 0.04550962 0.01781663 0.16937722 0.00189123
 0.03267109 0.02141926 0.00056773 0.00424576 0.01063301 0.02951516
 0.14269799 0.00078103 0.00731263 0.00182223 0.02955337 0.00566687
 0.01440053 0.07724281]


In [ ]:
debug_value([tf.exp(-tf.divide(topic_losses_recon, n_bow))])

### test

In [12]:
debug_shape([bow, hidden_bow, latents_bow, prob_topic, bow_embeddings, topic_embeddings, topic_bow, prob_bow])

Placeholder:0 : (32, 4022)
topic/enc/dropout/cond/Merge:0 : (32, 256)
topic/enc/add:0 : (32, 32)
topic/enc/prob/Softmax:0 : (32, 50)
embedding_lookup:0 : (4022, 256)
topic/dec/topic_emb:0 : (50, 256)
topic/dec/Softmax:0 : (50, 4022)
topic/dec/Log:0 : (32, 4022)


In [27]:
debug_shape([topic_losses_recon, topic_loss_recon, n_bow, ppls, topic_embeddings_norm, tf.expand_dims(topic_angles_mean, -1), topic_angles_vars])

Sum:0 : (32,)
Neg:0 : ()
Sum_2:0 : (32,)
Neg_1:0 : (32,)
truediv_1:0 : (50, 256)
ExpandDims_1:0 : (1,)
Mean_3:0 : ()


In [14]:
debug_value([tf.reduce_sum(tf.square(topic_embeddings_norm), 1)], return_value=True)[0]

array([1.        , 0.99999994, 0.9999999 , 1.        , 1.        ,
       1.0000001 , 0.9999999 , 1.0000001 , 1.0000001 , 1.        ,
       1.        , 1.        , 1.        , 1.        , 1.        ,
       1.        , 0.9999999 , 0.9999999 , 0.99999994, 1.        ,
       1.        , 0.9999999 , 1.0000001 , 1.        , 1.        ,
       1.        , 0.99999994, 1.        , 0.99999994, 0.99999994,
       1.0000001 , 0.9999999 , 1.        , 1.        , 1.        ,
       1.        , 1.        , 1.        , 0.9999999 , 1.        ,
       0.9999999 , 1.        , 1.        , 1.        , 0.99999994,
       1.0000001 , 1.        , 1.        , 0.99999994, 1.        ],
      dtype=float32)

In [15]:
debug_value([tf.reduce_sum(prob_topic, -1), tf.reduce_sum(topic_bow, -1), tf.reduce_sum(tf.exp(prob_bow), 1)])

Sum_4:0 : [1.0000001  1.         1.0000001  0.99999994 1.         1.
 1.         0.99999994 0.99999994 1.         1.         1.
 0.99999994 1.         1.         1.         1.         1.
 1.         1.         1.         1.         1.         0.9999999
 1.         0.99999994 1.         1.0000001  0.99999994 1.
 1.         1.        ]
Sum_5:0 : [1.         0.99999994 1.         0.99999994 0.99999994 1.
 1.         0.9999998  1.         1.         1.         1.
 1.         1.         1.         1.0000001  0.99999994 1.
 1.         0.99999994 0.9999999  0.99999994 1.0000001  1.
 1.         1.         1.         0.99999994 0.9999999  1.
 1.         1.         0.99999994 1.         0.99999994 1.
 1.         0.9999999  1.         1.         1.         1.
 0.99999994 1.         0.99999994 0.99999994 0.99999994 0.99999994
 1.         1.        ]
Sum_6:0 : [1.         1.         1.0000001  0.99999994 1.         1.
 0.9999999  0.99999994 0.9999999  1.         1.0000001  1.
 0.9999999  1.        

In [16]:
sigma_bow = tf.exp(0.5 * logvars_bow)
dist_bow = tfd.Normal(means_bow, sigma_bow)
dist_std = tfd.Normal(0., 1.)
topic_loss_kl_tmp = tf.reduce_mean(tf.reduce_sum(tfd.kl_divergence(dist_bow, dist_std), 1))

In [17]:
debug_value([topic_loss_recon, topic_loss_kl, topic_loss_kl_tmp])

Neg:0 : 405.38312
Mean_1:0 : 0.32056683
Mean_4:0 : 0.32056683


In [110]:
_logvars, _means, _kl_losses, _latents, _output_logits = sess.run([logvars, means, kl_losses, latents, output_logits], feed_dict=feed_dict)


In [111]:
_logvars.shape, _means.shape, _kl_losses.shape, _latents.shape

((32, 32), (32, 32), (32,), (32, 32))

In [112]:
_output_logits

array([[[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       ...,

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]]], dtype=float32)

In [109]:
_output_logits, _dec_target_idxs_do, _dec_mask_tokens_do, _recon_loss, _kl_losses, _ = sess.run([output_logits, dec_target_idxs_do, dec_mask_tokens_do, recon_loss, kl_losses, opt], feed_dict=feed_dict)


NameError: name 'dec_target_idxs_do' is not defined

In [44]:
tf.reduce_max(output_logits, 2).eval(session=sess, feed_dict=feed_dict).shape

(120, 46)

In [31]:
_output_logits.shape, _dec_target_idxs_do.shape, _dec_mask_tokens_do.shape

((120, 46, 20000), (120, 46), (120, 46))

In [32]:
_logits = np.exp(_output_logits) / np.sum(np.exp(_output_logits), 2)[:, :, None]

In [33]:
_idxs = _dec_target_idxs_do

In [35]:
_losses = np.array([[-np.log(_logits[i, j, _idxs[i, j]]) for j in range(_idxs.shape[1])] for i in range(_idxs.shape[0])]) * _dec_mask_tokens_do

In [36]:
np.sum(_losses)/np.sum(_dec_mask_tokens_do)

9.903732

In [37]:
_recon_loss

9.903732

In [38]:
_kl_losses.shape

(120,)